In [23]:
import pandas as pd

# Ustawienia wyświetlania
pd.set_option('display.max_rows', None)       # pokaż wszystkie wiersze
pd.set_option('display.max_columns', None)    # pokaż wszystkie kolumny
pd.set_option('display.width', None)          # brak ograniczenia szerokości
pd.set_option('display.max_colwidth', None)   # pełna szerokość kolumny (np. dla tekstu)


In [15]:
stock_list = pd.read_csv('stock_list.csv')
income = pd.read_csv('income')
market_val = pd.read_csv('market_value')
profitability = pd.read_csv('profitability')
cash_flow = pd.read_csv('cash_flow')
liabilities = pd.read_csv('liabilities')
liquidity = pd.read_csv('liquidity')
activity = pd.read_csv('activity')

In [16]:
# Łączenie DataFrame'ów po kolei bez użycia reduce
all_data = income.merge(market_val, on=['Nazwa', 'Ticker', 'Kwartały'], how='inner') \
                  .merge(profitability, on=['Nazwa', 'Ticker', 'Kwartały'], how='inner') \
                  .merge(cash_flow, on=['Nazwa', 'Ticker', 'Kwartały'], how='inner') \
                  .merge(liabilities, on=['Nazwa', 'Ticker', 'Kwartały'], how='inner') \
                  .merge(liquidity, on=['Nazwa', 'Ticker', 'Kwartały'], how='inner') \
                  .merge(activity, on=['Nazwa', 'Ticker', 'Kwartały'], how='inner')

# Podgląd wyniku
all_data.head()

,Nazwa,Ticker,Kwartały,Data,Przychody ze sprzedaży,Zysk ze sprzedaży,Zysk operacyjny (EBIT),Zysk z działalności gospodarczej,Zysk przed opodatkowaniem,Zysk netto,...,Rotacja należności,Cykl należności,Cykl zobowiązań,Rotacja zapasów,Cykl zapasów,Rotacja majątku obrotowego,Rotacja majątku trwałego,Rotacja majątku ogółem,Cykl operacyjny,Cykl konwersji gotówki
0,06MAGNA,06N,2005/Q4,2006-02-14,-9 702,-283,-283,-2 457,-2 457,-785,...,"0,42",868,2 920,"1,49",245,"0,04","0,07","0,03",1 113,-1 807
1,06MAGNA,06N,2006/Q1,2006-05-15,1 150,-667,-667,-667,-667,633,...,"-0,74",-496,-1 975,"-1,76",-207,"-0,06","-0,11","-0,04",-703,1 272
2,06MAGNA,06N,2006/Q2,2006-08-14,2 799,1 539,1 539,1 539,1 539,3 707,...,"0,12",3 118,17 994,"0,48",755,"0,01","0,02","0,00",3 873,-14 121
3,06MAGNA,06N,2006/Q3,2006-11-14,18 282,17 507,17 507,17 507,17 507,20 167,...,"1,58",230,2 194,"-1,02",-358,"0,05","0,21","0,04",-128,-2 322
4,06MAGNA,06N,2006/Q4,2007-03-01,-16 331,-25 673,-25 695,-25 695,-25 695,2 235,...,"0,95",385,4 607,"2,94",124,"0,03","0,18","0,02",509,-4 098


In [20]:
indicators = income[['Nazwa', 'Ticker', 'Kwartały', 'Data']].merge(market_val, on=['Nazwa', 'Ticker', 'Kwartały'], how='inner') \
                  .merge(profitability, on=['Nazwa', 'Ticker', 'Kwartały'], how='inner') \
                  .merge(liabilities, on=['Nazwa', 'Ticker', 'Kwartały'], how='inner')

indicators.head()

,Nazwa,Ticker,Kwartały,Data,Kurs,Wartość księgowa,Cena/WK,Zysk na akcję,Cena/Zysk,Przychód,...,Zadłużenie długoterminowe,Zadłużenie środków trwałych,Pokrycie aktywów trwałych kapitałami stałymi,Trwałość struktury finansowania,Zastosowanie kapitału obcego,Wskaźnik ogólnej sytuacji finansowej,Zadłużenie netto,Zadłużenie netto / EBITDA,Zadłużenie finansowe netto,Zadłużenie finansowe netto / EBITDA
0,06MAGNA,06N,2005/Q4,2006-02-14,"62,00","69,22","0,90","0,93","66,94","2,59",...,NaN,NaN,NaN,NaN,"0,35","0,71",-73 599 000,"90,31",-90 034 000,"110,47"
1,06MAGNA,06N,2006/Q1,2006-05-15,"69,00","69,50","0,99","0,86","79,89","-4,08",...,NaN,NaN,NaN,NaN,"0,33","0,67",-76 836 000,"30,49",-77 652 000,"30,81"
2,06MAGNA,06N,2006/Q2,2006-08-14,"76,00","70,47","1,08","1,14","66,93","0,47",...,NaN,NaN,NaN,NaN,"0,25","5,79",-63 315 000,"28,00",-138 888 000,"61,43"
3,06MAGNA,06N,2006/Q3,2006-11-14,"88,00","31,50","2,79","8,03","10,96","4,17",...,"0,05","4,04","4,93","0,41","0,38","1,77",46 372 000,"2,57",-24 249 000,"-1,34"
4,06MAGNA,06N,2006/Q4,2007-03-01,"97,60","30,73","3,18","7,65","12,77","1,96",...,NaN,NaN,NaN,NaN,"0,37","1 111,18",7 704 000,"-1,06",-60 059 000,"8,25"


In [25]:
income_yearly = income[income['Kwartały'].str.contains('Q4', na=False)]

income_yearly.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1620 entries, 3 to 10666
Data columns (total 11 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   Nazwa                             1620 non-null   object 
 1   Ticker                            1620 non-null   object 
 2   Kwartały                          1620 non-null   object 
 3   Data                              1620 non-null   object 
 4   Przychody ze sprzedaży            1571 non-null   object 
 5   Zysk ze sprzedaży                 1571 non-null   object 
 6   Zysk operacyjny (EBIT)            1571 non-null   object 
 7   Zysk z działalności gospodarczej  1571 non-null   object 
 8   Zysk przed opodatkowaniem         1571 non-null   object 
 9   Zysk netto                        1620 non-null   object 
 10  EBITDA                            0 non-null      float64
dtypes: float64(1), object(10)
memory usage: 151.9+ KB
